In [1]:
import pandas as pd
import numpy as np

In [2]:
#读取数据
data = pd.read_csv(r'D:\data\mushroom-classification\mushrooms.csv')
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
#对标称型数据数值化
n,m = data.shape
for i in range(m):
    data[data.columns[i]] = pd.factorize(data[data.columns[i]])[0]
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


In [4]:
#数据集划分
np.random.seed(1)
random_indexs = np.random.permutation(n)
train_index = random_indexs[:int(0.7*n)]
test_index = random_indexs[int(0.7*n):]
train_data = data.iloc[train_index]
train_labels = train_data['class']
train_data = train_data[train_data.columns[1:]]
test_data = data.iloc[test_index]
test_labels = test_data['class']
test_data = test_data[test_data.columns[1:]]

In [11]:
class TNode():
    def __init__(self, feature=None, value=None, is_leaf=-1):
        '''
        feature,value:树节点对应的特征以及特征取值
        left,right:左右子树
        is_leaf:当值为-1表示为非叶子结点，否则表示类别
        '''
        self.feature = feature
        self.value = value
        self.left = None
        self.right = None
        self.is_leaf = is_leaf
class CART():
    def __init__(self, data, labels, epsilon=0,min_samples=100):
        '''
        data,label:训练数据以及标签
        T:CART分类树根节点
        epsilon:gini阈值,当gini小于epsilon时停止分裂
        num_data，classn_num：训练样本个数，类的个数
        min_samples：最小样本数阈值，当当前节点样本数小于min_samples时停止分裂
        '''
        self.data = data
        self.labels = labels
        self.epsilon = epsilon
        self.num_data = data.shape[0]
        self.class_num = len(np.unique(labels))
        self.min_samples = min_samples
        self.T = self.create_tree(data,labels,features=list(range(data.shape[1])))

    def gini(self, data, labels, feature=None, value=None):
        #计算gini指数
        if data.shape[0] == 0:
            return 0
        if not feature and not value:
            p_cls = np.zeros(self.class_num)
            for c in range(self.class_num):
                c_num = (labels == c).sum()
                p_cls[c] = c_num / data.shape[0]
            res = 1 - (p_cls ** 2).sum()
        else:
            res = 0
            sub_index1 = np.where(self.data[:, feature] == value)
            sub_index2 = np.where(self.data[:, feature] != value)
            sub_data1 = self.data[sub_index1, :].reshape(-1,data.shape[1])
            sub_data2 = self.data[sub_index2, :].reshape(-1,data.shape[1])
            sub_labels1 = self.labels[sub_index1]
            sub_labels2 = self.labels[sub_index2]
            p1 = sub_data1.shape[0] / data.shape[0]
            p2 = 1 - p1
            res = (p1 * self.gini(sub_data1, sub_labels1) + p2 * self.gini(sub_data2, sub_labels2))
        return res

    def create_tree(self, data, labels, features):
        #构造CART分类决策树
        if data.shape[0] == 0:
            return None
        if data.shape[0] <= self.min_samples or len(features) == 0 or len(np.unique(labels)) == 1:
            cls = np.argmax(np.bincount(labels))
            T = TNode(is_leaf=cls)
            return T
        #存储所有特征及取值的对应gini指数
        fea_vals = {}
        for f in features:
            values = np.unique(data[:, f])
            d = {key: 0 for key in values}
            for v in d.keys():
                d[v] = self.gini(data, labels, f, v)
            fea_vals[f] = d
        min_gini = float('inf')
        min_fea = 0
        min_val = 0
        for fea,d in fea_vals.items():
            for val,gini in d.items():
                if gini < min_gini:
                    min_gini = gini
                    min_fea = fea
                    min_val = val
        if min_gini <= self.epsilon:
            cls = np.argmax(np.bincount(labels))
            T = TNode(is_leaf=cls)
            return T
        T = TNode(min_fea, min_val)
        sub_index1 = data[:, min_fea] == min_val
        sub_index2 = data[:, min_fea] != min_val
        sub_data1 = data[sub_index1, :]
        sub_data2 = data[sub_index2, :]
        sub_labels1 = labels[sub_index1]
        sub_labels2 = labels[sub_index2]
        features.remove(min_fea)
        T.feature = min_fea
        T.value = min_val
        T.left = self.create_tree(sub_data1, sub_labels1, features)
        T.right = self.create_tree(sub_data2, sub_labels2, features)
        return T

    def predict(self,x):
        T = self.T
        while T.is_leaf == -1:
            if x[T.feature] == T.value:
                T = T.left
            else:
                T = T.right
        return T.is_leaf

    def test(self,X,labels):
        correct = 0
        for i in range(X.shape[0]):
            pred = self.predict(X[i,:])
            if pred == labels[i]:
                correct += 1
        print('测试集正确率:%f' %(correct * 100 / X.shape[0]))

In [12]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)
model = CART(train_data,train_labels)
model.test(test_data,test_labels)

测试集正确率:94.708778
